# Introduction

## Goal
The goal of this lab is to study two of the most famous discrete discrete optimization benchmark problems - the Traveling Salesman Problem (TSP) and the Knapsack problem.

Discrete optimization problems often present difficulties for naïve Evolutionary Computation approaches, where special care must be taken to generate and maintain feasible solutions and/or to sufficiently penalize infeasible solutions. Here we will see how Ant Colony Optimization can solve this kind of discrete problems much more efficiently than Evolutionary Computation approaches. In addition to that, we will study the effect of parametrization on the performance of Ant Colony Optimization.

Note that, unless otherwise specified, in this module's exercises the aim of the algorithms will be to maximize the fitness function $f(x)$, i.e. higher values correspond to a better fitness!

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import collections as mc
from numpy.typing import NDArray
import numpy as np


def readFileAsMatrix(file: str) -> list[list[float]]:
    with open(file) as f:
        lines = f.read().splitlines()
        matrix: list[list[float]] = []
        for line in lines:
            row: list[float] = []
            for value in line.split():
                row.append(float(value))
            matrix.append(row)
        return matrix


def readFileAsList(file: str) -> NDArray[np.uint32]:
    with open(file) as f:
        lines = f.read().splitlines()
        n = len(lines)
        array = np.empty(n, dtype=np.uint32)
        for i in range(n):
            array[i] = int(lines[i])
        return array


def plotSolution(
    points: list[list[float]],
    distances: list[list[float]],
    solution: list[int],
    title: str,
):
    f, ax = plt.subplots(1, 1)
    f.suptitle(title)
    ax.scatter(*zip(*points))

    for i, p in enumerate(points):
        ax.annotate(str(i), (p[0], p[1]))

    # draw all possible path segments
    lines = []
    for i, p in enumerate(points):
        for j, _ in enumerate(points):
            if distances[i][j] > 0 and i > j:
                lines.append((points[i], points[j]))
    lc = mc.LineCollection(lines, linewidths=0.1)
    ax.add_collection(lc)

    # draw the solution
    lines = []
    for i in np.arange(len(solution) - 1):
        lines.append((points[solution[i]], points[solution[i + 1]]))
    lines.append((points[solution[0]], points[solution[-1]]))
    lc = mc.LineCollection(lines, linewidths=1, color="r")
    ax.add_collection(lc)

    # ax.set_title(title)
    ax.autoscale()
    ax.margins(0.1)

# Exercise 1

## Traveling Salesman Problem

In the Traveling Salesman Problem (TSP)$^{[1]}$, a salesperson is expected to make a round trip visiting a set of $N$ cities, returning the starting city. The goal is to minimize the total distance traveled. In the simplest version of the TSP, that we consider in this exercise, it is assumed that it is possible to travel from any city $i$ to any city $j$ (i.e., cities form a fully-connected graph), and the distance is the same both ways.
To specify a TSP instance, we need the spatial coordinates $(x, y)$ for each of the $N$ cities, and an adjacency matrix $(N × N)$ describing the pairwise distances between cities. These can be computed as Euclidean distances between cities, or in general can be specified according to a given distance function (which may take into account travel cost, time, and other factors). Candidate solutions for the TSP can be easily represented as permutations of the list of city indices (enumerating the order in which cities should be visited). For instance, if there are $5$ cities, then a candidate solution might be $[4, 1, 0, 2, 3]$ (assuming zero-indexing).

In case of Evolutionary Computation approaches, a simple sequence representation can be used, coupled with mechanisms that ensure that the candidate solutions remain feasible during crossover and mutation. This can be accomplished by ad hoc genetic operators (which in the inspyred framework are dubbed as "variators"), namely the partially matched crossover and the inversion mutation, as shown in the next cell. As an alternative, we can use Ant Colony Optimization that is naturally suited for solving graph-like problems such as the TSP and, differently from Evolutionary Computation approaches, does not require any specific operator. To start the experiments run the next cell $^{[2]}$.

The code will perform a single run of Ant Colony Systems (ACS) and a customized Evolutionary Algorithm (EA) and show you the fitness trends of both algorithms, as well as a map corresponding to the best solutions provided by the two algorithms. Note that for both algorithms the fitness function is defined as the reciprocal of the total distance traveled ($f(x) = \frac{1}{\sum{d}}$), to be maximized, such that maximizing $f(x)$ corresponds to minimizing the total distance.

Execute the following cell on different problem instances (this can be obtained by changing the variable "instance" in the next cell) and observe the behavior of the two algorithms on each instance. Note that the run time can be quite long (several seconds)
on larger instances (especially on *att48*). 

- Which algorithm provides the best solution in most cases? 
- What can you say about the number of function evaluations needed to converge?

---
[1]: See https://people.sc.fsu.edu/~jburkardt/datasets/tsp/tsp.html

[2]: For all the exercises in this lab you may set the seed for the pseudo-random number generator. This will allow you to reproduce your results.

[3]: Note that you can also create your own instances by editing the two variables "points" (that contains a set of coordinates
$(x, y)$) and "distances" (that contains an adjacency matrix that can be computed e.g. by Euclidean distances between points).

In [ ]:
from utils.plot_utils import plot_observer
from inspyred.ec import (
    variators,
    terminators,
    selectors,
    replacers,
    EvolutionaryComputation,
)
from inspyred.benchmarks import TSP
from inspyred.swarm import ACS
from random import Random

optimal_distances = {
    "p01": 291,
    "bays29": 2020,
    "att48": 10628,
}
instance = "p01"
# instance = "bays29"
# instance = "att48"
points = readFileAsMatrix("../../datasets/lab07/tsp/" + instance + "_xy.txt")
distances = readFileAsMatrix("../../datasets/lab07/tsp/" + instance + "_d.txt")

# common parameters
pop_size = 50
max_generations = 20
seed = 41
prng = Random(seed)
display = True
# ACS specific parameters
evaporation_rate = 0.1
learning_rate = 0.1
# EA specific parameters
tournament_size = 5
num_elites = 1

args = {}
args["fig_title"] = "ACS"

# run ACS
problem = TSP(distances)
ac = ACS(prng, problem.components)
ac.observer = [plot_observer]  # type: ignore
ac.terminator = terminators.generation_termination
final_pop = ac.evolve(
    generator=problem.constructor,
    evaluator=problem.evaluator,
    bounder=problem.bounder,
    maximize=problem.maximize,
    pop_size=pop_size,
    max_generations=max_generations,
    evaporation_rate=evaporation_rate,
    learning_rate=learning_rate,
    **args,
)
best_ACS = max(ac.archive)  # type: ignore

args["fig_title"] = "EA"

# run EA
problem = TSP(distances)
ea = EvolutionaryComputation(prng)
ea.observer = [plot_observer]  # type: ignore
ea.selector = selectors.tournament_selection
ea.variator = [  # type: ignore
    variators.partially_matched_crossover,
    variators.inversion_mutation,
]
ea.replacer = replacers.generational_replacement
ea.terminator = terminators.generation_termination
final_pop = ea.evolve(
    generator=problem.generator,
    evaluator=problem.evaluator,
    bounder=problem.bounder,
    maximize=problem.maximize,
    pop_size=pop_size,
    max_generations=max_generations,
    num_selected=pop_size,
    tournament_size=tournament_size,
    num_elites=num_elites,
    **args,
)
best_EA = max(ea.population)  # type: ignore

indices = []
for link in best_ACS.candidate:
    indices.append(link.element[0])
indices.append(best_ACS.candidate[-1].element[1])
solution_ACS = indices

solution_EA = best_EA.candidate

print(f"Optimal Distance: {optimal_distances[instance]}")
print(f"Distance: {1 / best_ACS.fitness} - Best Solution ACS: {solution_ACS}")
print(f"Distance: {1 / best_EA.fitness} - Best Solution EA: {solution_EA}")

plotSolution(points, distances, solution_ACS, "ACS (best solution)")
plotSolution(points, distances, solution_EA, "EA (best solution)")

# Exercise 2
## Knapsack problem 
In the Knapsack problem $^{[1]}$, we are given a knapsack of fixed capacity $C$. We are also given a list of $N$ items, each having a weight $w$ and a value $v$. We can put any subset of the items into the knapsack, as long as the total weight of the selected items does not exceed $C$. In the most general formulation of the problem, it is possible to select the same item multiple times. We aim to maximize the total value of the selection, which is the sum of the values of each item we put into the knapsack. Thus, a solution of the Knapsack problem is a subset $S$ of the $N$ items, each considered a certain number of times, for which the total weight is less than or equal to $C$, and which maximizes the total value. If the same item can be selected multiple times, the problem is called *Knapsack with duplicates*, otherwise if any item can be selected only once, the problem is called *0/1 Knapsack*.

Candidate solutions for the Knapsack problem can be represented as either a binary list (for the *0/1 Knapsack*) or as a list of non-negative integers (for the Knapsack with duplicates). In each case, the length of the list is the same as the number of items, and each element of the list corresponds to the quantity of the corresponding item to place in the knapsack. In case of Evolutionary Computation approaches, we can use as genetic operators uniform crossover and Gaussian mutation, as shown in the next cell. The reason we can use Gaussian mutation, even though the candidates are composed of discrete values, is because the bounder created by the Knapsack benchmark is an instance of $DiscreteBounder$, which automatically moves an illegal component to its nearest legal value (an integer, in this case). Once again, as an alternative we can use Ant Colony Optimization.
In this exercise, we will focus on the *0/1 Knapsack problem* (this is obtained by setting duplicates=False in next cell). To start the experiments, you need to run the next cell. The script will perform a single run of Ant Colony Systems (ACS) and a customized Evolutionary Algorithm (EA) and show you the fitness trends of both algorithms.

Execute the script on different problem instances (this can be obtained by changing the variable "instance" in the next cell) and observe the behavior of the two algorithms on each instance $^{[2]}$. Note that the run time can be quite long (several seconds) on larger instances. 

- Which algorithm provides the best solution in most cases?
- What can you say about the number of function evaluations needed to converge?

---
[1]: See https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/knapsack_01.html

[2]: Note that you can also create your own instances by editing the two variables "items" (a set of pairs $<w, v>$)
and "capacity". Both variables must be expressed as integer values.


In [ ]:
from utils.plot_utils import plot_observer
from itertools import compress
from inspyred.benchmarks import Knapsack
from inspyred.swarm import ACS
from inspyred.ec import (
    EvolutionaryComputation,
    selectors,
    replacers,
    terminators,
    variators,
)
import numpy as np
from random import Random

instance = "01"
# instance = "02"
# instance = "03"
# instance = "04"
# instance = "05"
# instance = "06"
# instance = "07"
# instance = "08"
capacity = readFileAsList("../../datasets/lab07/knapsack_01/p" + instance + "_c.txt")[0]
weights = readFileAsList("../../datasets/lab07/knapsack_01/p" + instance + "_w.txt")
values = readFileAsList("../../datasets/lab07/knapsack_01/p" + instance + "_p.txt")
solution = readFileAsList("../../datasets/lab07/knapsack_01/p" + instance + "_s.txt")
items = [(w, v) for w, v in zip(weights, values)]

# common parameters
pop_size = 50
max_generations = 20
duplicates = False
seed = 41
prng = Random(seed)
display = True
# ACS specific parameters
evaporation_rate = 0.1
learning_rate = 0.1
# EA specific parameters
tournament_size = 5
num_elites = 1

args = {}
args["fig_title"] = "ACS"

# run ACS
problem = Knapsack(capacity, items, duplicates=duplicates)
ac = ACS(prng, problem.components)
ac.observer = [plot_observer]  # type: ignore
ac.terminator = terminators.generation_termination
final_pop = ac.evolve(
    generator=problem.constructor,
    evaluator=problem.evaluator,
    bounder=problem.bounder,
    maximize=problem.maximize,
    pop_size=pop_size,
    max_generations=max_generations,
    evaporation_rate=evaporation_rate,
    learning_rate=learning_rate,
    **args,
)
best_ACS = max(ac.archive)  # type: ignore

args["fig_title"] = "EA"

# run EA
problem = Knapsack(capacity, items, duplicates=duplicates)
ea = EvolutionaryComputation(prng)
ea.observer = [plot_observer]  # type: ignore
ea.selector = selectors.tournament_selection
ea.variator = [  # type: ignore
    variators.uniform_crossover,
    variators.gaussian_mutation,
]
ea.replacer = replacers.generational_replacement
ea.terminator = terminators.generation_termination
final_pop = ea.evolve(
    generator=problem.generator,
    evaluator=problem.evaluator,
    bounder=problem.bounder,
    maximize=problem.maximize,
    pop_size=pop_size,
    max_generations=max_generations,
    num_selected=pop_size,
    tournament_size=tournament_size,
    num_elites=num_elites,
    **args,
)
best_EA = max(ea.population, key=lambda x: x.fitness)  # type: ignore

indices = []
for item in best_ACS.candidate:
    index = items.index((item.element, item.value))
    indices.append(index)
solution_ACS = np.zeros(len(items), dtype=np.uint16)
for i in indices:
    solution_ACS[i] += 1
solution_ACS = solution_ACS.tolist()

solution_EA = best_EA.candidate

used_capacity_aco = sum(list(compress(weights, solution_ACS)))
used_capacity_ea = sum(list(compress(weights, solution_EA)))

print("Capacity: ", capacity)
print(
    f"Optimal Solution:  {solution.tolist()} - Fitness value: {sum([values[i] for i in range(len(solution)) if solution[i] == 1])}"
)
print(
    f"Best Solution ACS: {solution_ACS} - Fitness value: {best_ACS.fitness}, - Unused capacity: {capacity - used_capacity_aco}"
)
print(
    f"Best Solution EA : {solution_EA} - Fitness value: {best_EA.fitness} - Unused capacity: {capacity - used_capacity_ea}"
)

# Exercise 3

In this exercise we will focus on the *Knapsack with duplicates* (this is obtained by setting duplicates=True in the next cell). To start the experiments, you need to run the next cell. Once again, the script will perform a single run of Ant Colony Systems (ACS) and a customized Evolutionary Algorithm (EA) and show you the fitness trends of both algorithms.

Execute the script on different problem instances (this can be obtained by changing the variable "instance" in the next cell) and observe the behavior of the two algorithms on each instance $^{[1]}$. Note that the run time can be quite long (several seconds) on larger instances.

- Which algorithm provides the best solution in most cases?
- What can you say about the number of function evaluations needed to converge? 
- Do you observe any difference on the algorithmic behavior between this exercise and the previous one?

---
[1]: Also in this case you can also create your own instances by editing the two variables "items" and "capacity".

In [ ]:
from utils.plot_utils import plot_observer
from inspyred.benchmarks import Knapsack
from inspyred.swarm import ACS
from inspyred.ec import (
    EvolutionaryComputation,
    selectors,
    replacers,
    terminators,
    variators,
)
import numpy as np
from random import Random

# datasets taken from https://people.sc.fsu.edu/~jburkardt/datasets/knapsack_01/knapsack_01.html
instance = "01"
# instance = "02"
# instance = "03"
# instance = "04"
# instance = "05"
# instance = "06"
# instance = "07"
# instance = "08"
capacity = readFileAsList("../../datasets/lab07/knapsack_01/p" + instance + "_c.txt")[0]
weights = readFileAsList("../../datasets/lab07/knapsack_01/p" + instance + "_w.txt")
values = readFileAsList("../../datasets/lab07/knapsack_01/p" + instance + "_p.txt")
items = [(w, v) for w, v in zip(weights, values)]

# common parameters
pop_size = 50
max_generations = 50
duplicates = True
seed = 41
prng = Random(seed)
display = True
# ACS specific parameters
evaporation_rate = 0.1
learning_rate = 0.1
# EA specific parameters
tournament_size = 2
num_elites = 0  # in general Ea tends to perform better when being less selective (allow more exploration) with instance 08

args = {}
args["fig_title"] = "ACS"

# run ACS
problem = Knapsack(capacity, items, duplicates=duplicates)
ac = ACS(prng, problem.components)
ac.observer = [plot_observer]  # type: ignore
ac.terminator = terminators.generation_termination
final_pop = ac.evolve(
    generator=problem.constructor,
    evaluator=problem.evaluator,
    bounder=problem.bounder,
    maximize=problem.maximize,
    pop_size=pop_size,
    max_generations=max_generations,
    evaporation_rate=evaporation_rate,
    learning_rate=learning_rate,
    **args,
)
best_ACS = max(ac.archive)  # type: ignore

args["fig_title"] = "EA"

# run EA
problem = Knapsack(capacity, items, duplicates=duplicates)
ea = EvolutionaryComputation(prng)
ea.observer = [plot_observer]  # type: ignore
ea.selector = selectors.tournament_selection
ea.variator = [  # type: ignore
    variators.uniform_crossover,
    variators.gaussian_mutation,
]
ea.replacer = replacers.generational_replacement
ea.terminator = terminators.generation_termination
final_pop = ea.evolve(
    generator=problem.generator,
    evaluator=problem.evaluator,
    bounder=problem.bounder,
    maximize=problem.maximize,
    pop_size=pop_size,
    max_generations=max_generations,
    num_selected=pop_size,
    tournament_size=tournament_size,
    num_elites=num_elites,
    **args,
)
best_EA = max(ea.population)  # type: ignore

indices = []
for item in best_ACS.candidate:
    index = items.index((item.element, item.value))
    indices.append(index)
solution_ACS = np.zeros(len(items), dtype=np.uint16)
for i in indices:
    solution_ACS[i] += 1
solution_ACS = solution_ACS.tolist()

solution_EA = best_EA.candidate

used_capacity_aco = 0
for i, item in enumerate(solution_ACS):
    if weights[i] != 0:
        used_capacity_aco += item * weights[i]

used_capacity_ea = 0
for i, item in enumerate(solution_EA):
    if item != 0:
        used_capacity_ea += item * weights[i]

print("Capacity: ", capacity)
print(
    f"Optimal Solution:  {solution.tolist()} - Fitness value: {sum([values[i] for i in range(len(solution)) if solution[i] == 1])}"
)
print(
    f"Best Solution ACS: {solution_ACS} - Fitness value: {best_ACS.fitness}, - Unused capacity: {capacity - used_capacity_aco}"
)
print(
    f"Best Solution EA : {solution_EA} - Fitness value: {best_EA.fitness} - Unused capacity: {capacity - used_capacity_ea}"
)

## Instruction and questions
Concisely note down your observations from the previous exercises (follow the bullet points) and think about the following questions. 
- What are the main differences between continuous and discrete optimization problems? Do you think that any of these two classes of problems is more difficult than the other?
- Why is ACS particularly suited for discrete optimization?
- Consider the two versions of the Knapsack problem (0/1, and with duplicates). Which of the two problems is more challenging from an optimization point of view? Why?